# Goal: Create the correlation matrix

This cell is for importing necessary modules.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import heapq # for minimum spanning tree

This cell is for reading the data file.

In [2]:
PSE_data = pd.read_csv(
    "daily.csv"
)
PSE_data['date_id'] = pd.to_datetime(PSE_data['date_id'])
PSE_data = PSE_data.set_index("date_id")
df_filtered = PSE_data.drop(columns=[col for col in PSE_data.columns if not (col.startswith('PH_') and col.endswith('_P')) and col != 'date_id'])

This code is for removing the other rows that are not within the last 10 years of the data. I also used the date September 25, 2013 as the start date so that when I calculate log returns, I will have exactly 10 years worth of data. 

In [3]:
df_filtered = df_filtered.loc['2013-09-25':'2023-09-26']

## Cleaning data
This block is for cleaning the data and removing columns with too many null values.

In [4]:
def has_high_null_percentage(column, null_threshold):
    return column.isnull().mean() > null_threshold

# def has_consecutive_constant_values(column, threshold):
#     constant_streak = column.groupby((column != column.shift()).cumsum()).transform('size')
#     return constant_streak.max() > threshold

# def has_end_constant_values(column, end_threshold):
#     end_section = column.iloc[-int(len(column) * 0.05):]
#     return end_section.nunique() == 1

# def has_few_variations(column, variation_threshold):
#     return column.nunique() < variation_threshold

null_percentage_threshold = round(df_filtered.shape[0] * 0.85)
# consecutive_constant_threshold = int(len(df_filtered) * 0.35) 
# end_constant_threshold = 0.01
# variation_threshold = 50

columns_to_drop = []

for col in df_filtered.columns:
    if col == "date_id":  
        continue
    
    if (
        has_high_null_percentage(df_filtered[col], null_percentage_threshold) # or
        # has_consecutive_constant_values(df_filtered[col], consecutive_constant_threshold) or
        # has_end_constant_values(df_filtered[col], end_constant_threshold) or
        # has_few_variations(df_filtered[col], variation_threshold)
    ):
        columns_to_drop.append(col)

df_filtered = df_filtered.drop(columns=columns_to_drop)

In [5]:
df_filtered

,PH_PIP_P,PH_ASA_P,PH_ABS_P,PH_AGN_P,PH_APC_P,PH_CHP_P,PH_CEU_P,PH_CIR_P,PH_CAA_P,PH_EEQ_P,...,PH_MJC_P,PH_PCK_P,PH_MRP_P,PH_LOT_P,PH_BAG_P,PH_H2O_P,PH_PRC_P,PH_SSN_P,PH_SIN_P,PH_SHK_P
date_id,,,,,,,,,,,,,,,,,,,,,
2013-09-25,NaN,7.0,33.0,6.0,1.0,NaN,11.0,12.0,46.0,10.0,...,1.0,6.0,11.0,6.0,13.0,5.0,10.0,12.0,0.0,NaN
2013-09-26,NaN,6.0,33.0,6.0,1.0,NaN,11.0,12.0,46.0,10.0,...,1.0,6.0,11.0,6.0,12.0,6.0,10.0,11.0,0.0,NaN
2013-09-27,NaN,6.0,33.0,6.0,1.0,NaN,11.0,12.0,46.0,10.0,...,1.0,6.0,11.0,6.0,13.0,5.0,10.0,11.0,0.0,NaN
2013-09-30,NaN,6.0,33.0,6.0,1.0,NaN,11.0,12.0,46.0,10.0,...,1.0,6.0,11.0,6.0,12.0,5.0,10.0,11.0,0.0,NaN
2013-10-01,NaN,7.0,33.0,6.0,1.0,NaN,11.0,12.0,46.0,10.0,...,1.0,6.0,10.0,6.0,13.0,5.0,10.0,11.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-20,2.0,11.0,3.0,3.0,0.0,1.0,8.0,2.0,44.0,5.0,...,1.0,4.0,7.0,4.0,5.0,1.0,7.0,2.0,1.0,9.0
2023-09-21,2.0,11.0,3.0,3.0,0.0,1.0,8.0,2.0,44.0,5.0,...,1.0,4.0,7.0,4.0,5.0,1.0,7.0,2.0,1.0,9.0
2023-09-22,2.0,11.0,3.0,3.0,0.0,1.0,8.0,2.0,44.0,5.0,...,1.0,4.0,7.0,3.0,5.0,1.0,7.0,2.0,1.0,9.0


## Calculate returns from raw prices; log returns are probably easier, but you can try others like standardized returns if you have time

This cell is for calculating log returns.

In [6]:
PSE_log_returns = df_filtered.copy()
numeric_cols = PSE_log_returns.columns.difference(["date_id"])
PSE_log_returns[numeric_cols] = PSE_log_returns[numeric_cols].replace(0, np.nan)
PSE_log_returns[numeric_cols] = np.log(PSE_log_returns[numeric_cols] / PSE_log_returns[numeric_cols].shift(1))

I removed the the row for September 25, 2013, and I filled all the nan values with zero.

In [7]:
PSE_log_returns = PSE_log_returns.loc['2013-09-26':'2023-09-26']
PSE_log_returns.columns = PSE_log_returns.columns.str.replace(r"_P", "").str.replace(r"PH_", "")

# this was for testing
# least_null_cols = PSE_log_returns.isna().sum().nsmallest(3).index
# trial = PSE_log_returns[least_null_cols]
# trial

## Finding an appropriate window size and creating a covariance matrix

The cell below is for creating the covariance matrix. I used a window size of 6 months.

In [8]:
# covariance_dict = {}
# window_size = 120

# for col in trial.columns:
#     for other in trial.columns:
#         if col != other:
#             covariances = []
#             A = trial[col].to_numpy()
#             B = trial[other].to_numpy()
            
#             for i in range(len(A) - window_size + 1):
#                 win_A = A[i:i + window_size]
#                 win_B = B[i:i + window_size]
                
#                 covar = np.cov(A, B)
#                 covariances.append(covar[0, 1])
                
#             covariance_dict[f"{col}_{other}"] = covariances

In [9]:
# code is from: https://youtu.be/oRbESNj83mY?si=5qT_HAsnko5lPIFJ

covar_matrix = PSE_log_returns.rolling(window=55).cov()
covar_matrix

PHIP  ASA       ABS       AGN  APC  CHP           CEU  \
date_id                                                                  
2013-09-26 PHIP   NaN  NaN       NaN       NaN  NaN  NaN           NaN   
           ASA    NaN  NaN       NaN       NaN  NaN  NaN           NaN   
           ABS    NaN  NaN       NaN       NaN  NaN  NaN           NaN   
           AGN    NaN  NaN       NaN       NaN  NaN  NaN           NaN   
           APC    NaN  NaN       NaN       NaN  NaN  NaN           NaN   
...               ...  ...       ...       ...  ...  ...           ...   
2023-09-26 H2O    0.0  0.0  0.000000  0.000000  NaN  0.0  0.000000e+00   
           PHRC   0.0  0.0  0.000036  0.001426  NaN  0.0 -6.930625e-06   
           SSN    0.0  0.0  0.000000  0.000000  NaN  0.0  0.000000e+00   
           SIN    0.0  0.0  0.000000  0.000000  NaN  0.0  0.000000e+00   
           SHK    0.0  0.0  0.000871 -0.000356  NaN  0.0 -1.559866e-20   

                      CIR       CAA  EEQ  ...  MJC          PHCK  MRP  \
date_id                                   ...                           
2013-09-26 PHIP       NaN       NaN  NaN  ...  NaN           NaN  NaN   
           ASA        NaN       NaN  NaN  ...  NaN           NaN  NaN   
           ABS        NaN       NaN  NaN  ...  NaN           NaN  NaN   
           AGN        NaN       NaN  NaN  ...  NaN           NaN  NaN   
           APC        NaN       NaN  NaN  ...  NaN           NaN  NaN   
...                   ...       ...  ...  ...  ...           ...  ...   
2023-09-26 H2O   0.000000  0.000000  0.0  ...  0.0  0.000000e+00  0.0   
           PHRC  0.000021 -0.000015  0.0  ...  0.0 -7.418235e-04  0.0   
           SSN   0.000000  0.000000  0.0  ...  0.0  0.000000e+00  0.0   
           SIN   0.000000  0.000000  0.0  ...  0.0  0.000000e+00  0.0   
           SHK  -0.000791 -0.000115  0.0  ...  0.0  2.606683e-20  0.0   

                          LOT  BAG  H2O          PHRC  SSN  SIN           SHK  
date_id                                                                        
2013-09-26 PHIP           NaN  NaN  NaN           NaN  NaN  NaN           NaN  
           ASA            NaN  NaN  NaN           NaN  NaN  NaN           NaN  
           ABS            NaN  NaN  NaN           NaN  NaN  NaN           NaN  
           AGN            NaN  NaN  NaN           NaN  NaN  NaN           NaN  
           APC            NaN  NaN  NaN           NaN  NaN  NaN           NaN  
...                       ...  ...  ...           ...  ...  ...           ...  
2023-09-26 H2O   0.000000e+00  0.0  0.0  0.000000e+00  0.0  0.0  0.000000e+00  
           PHRC  4.918081e-03  0.0  0.0  1.197040e-02  0.0  0.0 -1.800733e-20  
           SSN   0.000000e+00  0.0  0.0  0.000000e+00  0.0  0.0  0.000000e+00  
           SIN   0.000000e+00  0.0  0.0  0.000000e+00  0.0  0.0  0.000000e+00  
           SHK  -8.097096e-20  0.0  0.0 -1.800733e-20  0.0  0.0  2.055711e-03  

[767046 rows x 294 columns]

### Prof Jose

boolean arrays, less for loops <br>
covariance cant have null values nanstd <br>
masked array<br>
dendrogram <br>

In [10]:
A = np.array([1, 2, 3, 4, 5, 6, np.nan])

In [11]:
print(np.std(A))
print(np.max(A))
print(np.mean(A))

nan
nan
nan


In [12]:
print(np.nanstd(A))
print(np.nanmax(A))
print(np.nanmean(A))

1.707825127659933
6.0
3.5


In [13]:
# masked array
A[A > 3]

array([4., 5., 6.])

In [14]:
A > 3

array([False, False, False,  True,  True,  True, False])

# Goal:  Create the correlation and distance matrices for returns of PSE stocks over different windows and create the Minimum Spanning Tree

## Turn the covariance matrix into correlation matrix. Turn the correlation matrix into a distance matrix. Create the Minimum Spanning Tree. 

In [15]:
"""
# scratch code
def covar_to_correl(cov_matrix):
    # cov_array = cov_matrix.to_numpy() if isinstance(cov_matrix, pd.DataFrame) else cov_matrix
    cov_matrix = cov_matrix.apply(pd.to_numeric, errors='coerce')
    std_devs = np.sqrt(np.diag(cov_matrix))
    
    with np.errstate(invalid='ignore'):
        correlation_matrix = cov_matrix / np.outer(std_devs, std_devs)

    np.fill_diagonal(correlation_matrix.values, 1)
    
    return correlation_matrix

"""

"\n# scratch code\ndef covar_to_correl(cov_matrix):\n    # cov_array = cov_matrix.to_numpy() if isinstance(cov_matrix, pd.DataFrame) else cov_matrix\n    cov_matrix = cov_matrix.apply(pd.to_numeric, errors='coerce')\n    std_devs = np.sqrt(np.diag(cov_matrix))\n    \n    with np.errstate(invalid='ignore'):\n        correlation_matrix = cov_matrix / np.outer(std_devs, std_devs)\n\n    np.fill_diagonal(correlation_matrix.values, 1)\n    \n    return correlation_matrix\n\n"

In [16]:
 # from: https://gist.github.com/wiso/ce2a9919ded228838703c1c7c7dad13b
def covar_to_correl(cov_matrix):
    std_devs = np.sqrt(np.abs(np.diag(cov_matrix)))  # Standard deviations
    outer_v = np.outer(std_devs, std_devs)  # Outer product for normalization
    correlation = cov_matrix / outer_v  # Convert to correlation
    correlation[cov_matrix == 0] = 0  # Handle zeros
    np.fill_diagonal(correlation.values, 1)  # Ensure diagonal is 1
    return correlation

# from: https://www.geeksforgeeks.org/prims-minimum-spanning-tree-mst-greedy-algo-5/
def prim_mst(distance_matrix):
    if not isinstance(distance_matrix, pd.DataFrame):
        return []  # Or raise a TypeError if you prefer

    n = len(distance_matrix.index)
    if n == 0:  # Handle empty DataFrame case
        return []

    start_node = distance_matrix.index[0]  # Start with the first node
    visited = {start_node}
    mst_edges = []
    min_heap = []

    # Add initial edges from the start node to the heap
    for neighbor in distance_matrix.columns:
        if neighbor != start_node and not np.isnan(distance_matrix.loc[start_node, neighbor]):
            weight = distance_matrix.loc[start_node, neighbor]
            heapq.heappush(min_heap, (weight, start_node, neighbor))

    while min_heap:
        weight, u, v = heapq.heappop(min_heap)

        if v not in visited:
            visited.add(v)
            mst_edges.append((u, v, weight))

            for neighbor in distance_matrix.columns:
                if neighbor != v and neighbor not in visited and not np.isnan(distance_matrix.loc[v, neighbor]):
                    weight = distance_matrix.loc[v, neighbor]
                    heapq.heappush(min_heap, (weight, v, neighbor))

    return mst_edges

In [ ]:
skips = len(covar_matrix.columns)

for i in range(0, len(covar_matrix.index), skips):
    # this is to get the covar matrix
    covar1 = covar_matrix.iloc[i:i+skips]
    
    # this is to make the correlation matrix 
    correl_matrix = covar_to_correl(covar1)
    
    # this is to make the distance matrix
    distance_matrix = 1 - np.square(correl_matrix)

    # this is to create the MST
    # d = distance_matrix.reset_index()
    # to_dist = d.iloc[:, 1:]
    
    # if to_dist.isnull().values.any():
    #     continue
    # else:
    #     mst = prim_mst(to_dist)
        
    #     if mst:
    #         print("Minimum Spanning Tree Edges:")
    #         for u, v, weight in mst:
    #             print(f"({u}, {v}): {weight}")
    #     else:
    #         print("Could not compute MST. Check the input distance matrix.")